In [1]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split

from sklearn import set_config
set_config(display="diagram")

import warnings
warnings.filterwarnings('ignore')

### Project Setup

In [2]:
df_initial = pd.read_csv('files_for_lab/we_fn_use_c_marketing_customer_value_analysis.csv')
df_initial.head(2)

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,...,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,...,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,...,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize


### Data Cleaning

- create custom class for basic cleaning 
- create custome class for outlier removal
- create cleaning steps: apply basic cleaning and outlier removal to dataset

In [3]:
# Custom class for the initial cleaning steps of:
# removing duplicates, drop features, deal with date fatures and standardize headers

class CustomInitialDataCleaner:
    def __init__(self, drop_features=[], date_features=[]):
        self.drop_features = drop_features
        self.date_features = date_features
    
    def __remove_duplicates(self, X):
        return X[~X.duplicated()]
    
    def __drop_features(self, X):
        return X.drop(self.drop_features, axis=1, inplace=False)
    
    def __standardize_feature_headers(self, X):
        X.columns = [col.lower().replace(' ', '_') for col in X.columns]
        return X
    
    def __handle_dates(self, X):
        for date_f in self.date_features:
            date_feature = pd.to_datetime(X[date_f])
            X[f'{date_f}_year'] = date_feature.dt.year
            X[f'{date_f}_month'] = date_feature.dt.month
            X[f'{date_f}_day'] = date_feature.dt.day
            X.drop(date_f, axis=1, inplace=True)
            return X
        
    def clean(self, X):
        X = self.__remove_duplicates(X)
        X = self.__drop_features(X)
        X = self.__handle_dates(X)
        return self.__standardize_feature_headers(X)

In [4]:
# Custom class to deal with outliers
# IMPORTANT: this class removes rows where outliers are detected

class CustomOutlierRemover:
    def __init__(self, outlier_threshold, skip_features=[]):
        self.outlier_threshold = outlier_threshold
        self.skip_features = skip_features
        
    def __outliers_detect(self, X):
        for column in X.select_dtypes(np.number).columns:
            if column not in self.skip_features:
                upper = np.percentile(X[column],75)
                lower = np.percentile(X[column],25)
                iqr = upper - lower
                upper_limit = upper + self.outlier_threshold * iqr
                lower_limit = lower - self.outlier_threshold * iqr
                filtered = X[(X[column]>lower_limit) & (X[column]<upper_limit)]
                if filtered.shape[0] != X.shape[0]:
                    self.detected_outliers_.append({
                        'column': column, 
                        'ul_ll': [upper_limit, lower_limit], 
                        'removed_elements': X.shape[0] - filtered.shape[0] 
                    })
                    self.count_total_outliers_ += X.shape[0] - filtered.shape[0]
        
    def __outliers_remove(self, X):
        for outlier in self.detected_outliers_:
            filtered = X[(X[outlier['column']] > outlier['ul_ll'][1]) & (X[outlier['column']] < outlier['ul_ll'][0])]  
            df_removed_outliers = filtered
        return df_removed_outliers
    
    def clean(self, X):
        self.detected_outliers_ = []
        self.count_total_outliers_ = 0
        self.__outliers_detect(X)
        X_new = self.__outliers_remove(X)
        return X_new

In [7]:
outliers_skip_features = ['number_of_open_complaints','number_of_policies', \
                          'effective_to_date_year', 'effective_to_date_month', 'effective_to_date_day']

basicCleaner = CustomInitialDataCleaner(drop_features=['Customer'], date_features=['Effective To Date'])
outliersCleaner = CustomOutlierRemover(outlier_threshold=1.5, skip_features=outliers_skip_features)

df = basicCleaner.clean(df_initial)
print(f'df shape before outliers removal: {df.shape}')
df = outliersCleaner.clean(df.copy())
print(f'df shape after outliers removal: {df.shape}')
print(outliersCleaner.detected_outliers_)
print(f'removed lines: {outliersCleaner.count_total_outliers_}')

df shape before outliers removal: (9134, 25)
df shape after outliers removal: (8681, 25)
[{'column': 'customer_lifetime_value', 'ul_ll': [16414.039911750002, -3457.6210762500014], 'removed_elements': 817}, {'column': 'monthly_premium_auto', 'ul_ll': [170.5, 6.5], 'removed_elements': 430}, {'column': 'total_claim_amount', 'ul_ll': [960.3997301249999, -140.62664687499995], 'removed_elements': 453}]
removed lines: 1700


### Check Null Values

In [8]:
df.isna().sum()

state                            0
customer_lifetime_value          0
response                         0
coverage                         0
education                        0
employmentstatus                 0
gender                           0
income                           0
location_code                    0
marital_status                   0
monthly_premium_auto             0
months_since_last_claim          0
months_since_policy_inception    0
number_of_open_complaints        0
number_of_policies               0
policy_type                      0
policy                           0
renew_offer_type                 0
sales_channel                    0
total_claim_amount               0
vehicle_class                    0
vehicle_size                     0
effective_to_date_year           0
effective_to_date_month          0
effective_to_date_day            0
dtype: int64

### Pipeline Setup

#### Setup all parameters to define a custom pipeline which:
<br>
    0) since no null values found -> no step is worked in here to deal with them<br>
    2) applies StandardScaler to numerical features<br>
    3) applies OrdinalEncoder to selected categorcial features<br>
    4) applies OneHotEncoder to selected categorical features<br>
    5) outputs the prepared data set which can be then fed into a model<br>

#### Define ordinal features hierarchy and split the features 

In [10]:
# split features into numerical and remove target variable 
numerical = df.select_dtypes('number').drop('total_claim_amount', axis=1, inplace=False)
numerical_features = numerical.columns

# split features into categorical; then differentiate between ordinal categoricals and nominal ones
categorical = df.select_dtypes('object')
cat_ordinal_features = categorical[['coverage', 'location_code', 'employmentstatus', 'vehicle_size']].columns
cat_onehot_features = categorical.loc[:, ~categorical.columns.isin(cat_ordinal_features)].columns

# define order for the categorical features which will be handled by OerdinalEncoder
predefined_order = [['Premium', 'Extended', 'Basic'], ['Urban', 'Suburban', 'Rural'], \
            ['Employed', 'Medical Leave','Unemployed', 'Retired', 'Disabled'], ['Large', 'Medsize', 'Small']]


#### Define the pipelines and combine them into one data preparation pipeline

In [11]:
numerical_pipeline = Pipeline([
    ('standardize', StandardScaler()) 
])

cat_onehot_pipeline = Pipeline([
    ('one_hot_encoding', OneHotEncoder(sparse=False))
])

cat_ordinal_pipeline = Pipeline([
    ('ordinal_encoding', OrdinalEncoder(categories=predefined_order))
])

# final composite pipeline which will prepare the data for a model
data_preprocessing_pipeline = ColumnTransformer([
        ('numerical', numerical_pipeline, numerical_features),
        ('cat_ordinal', cat_ordinal_pipeline, cat_ordinal_features),
        ('cat_onehot', cat_onehot_pipeline, cat_onehot_features)
    ])

data_preprocessing_pipeline

ColumnTransformer(transformers=[('numerical',
                                 Pipeline(steps=[('standardize',
                                                  StandardScaler())]),
                                 Index(['customer_lifetime_value', 'income', 'monthly_premium_auto',
       'months_since_last_claim', 'months_since_policy_inception',
       'number_of_open_complaints', 'number_of_policies',
       'effective_to_date_year', 'effective_to_date_month',
       'effective_to_date_day'],
      dtype=...
                                                                             ['Large',
                                                                              'Medsize',
                                                                              'Small']]))]),
                                 Index(['coverage', 'location_code', 'employmentstatus', 'vehicle_size'], dtype='object')),
                                ('cat_onehot',
                                 Pipeline(steps=[('one_hot_encoding',
                                                  OneHotEncoder(sparse=False))]),
                                 Index(['state', 'response', 'education', 'gender', 'marital_status',
       'policy_type', 'policy', 'renew_offer_type', 'sales_channel',
       'vehicle_class'],
      dtype='object'))])

#### Split data train / test and apply preprocessing pipeline

In [12]:
# X-y split
y = df['total_claim_amount']
X = df.drop('total_claim_amount', axis=1, inplace=False)

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [13]:
# Fit the pipeline only on X_train to avoid data leakage
data_preprocessing_pipeline.fit(X_train)

ColumnTransformer(transformers=[('numerical',
                                 Pipeline(steps=[('standardize',
                                                  StandardScaler())]),
                                 Index(['customer_lifetime_value', 'income', 'monthly_premium_auto',
       'months_since_last_claim', 'months_since_policy_inception',
       'number_of_open_complaints', 'number_of_policies',
       'effective_to_date_year', 'effective_to_date_month',
       'effective_to_date_day'],
      dtype=...
                                                                             ['Large',
                                                                              'Medsize',
                                                                              'Small']]))]),
                                 Index(['coverage', 'location_code', 'employmentstatus', 'vehicle_size'], dtype='object')),
                                ('cat_onehot',
                                 Pipeline(steps=[('one_hot_encoding',
                                                  OneHotEncoder(sparse=False))]),
                                 Index(['state', 'response', 'education', 'gender', 'marital_status',
       'policy_type', 'policy', 'renew_offer_type', 'sales_channel',
       'vehicle_class'],
      dtype='object'))])

In [14]:
X_train_prepared = data_preprocessing_pipeline.transform(X_train)
X_test_prepared = data_preprocessing_pipeline.transform(X_test)

In [15]:
X_train_prepared

array([[ 2.46956407,  1.60016881, -0.88635949, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47115337, -1.27566083, -0.40654095, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.00337613, -0.43466996, -0.84945037, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.7686992 ,  1.48025528, -0.7018139 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.96409364,  1.76620548,  0.88527822, ...,  0.        ,
         0.        ,  1.        ],
       [-0.58505871,  1.21861367,  0.44236879, ...,  0.        ,
         0.        ,  0.        ]])

In [16]:
print(X_train.shape)
print(X_train_prepared.shape)


(6076, 24)
(6076, 57)


In [ ]:
# Now the model can be fitted with the prepared data ...